In [1]:
# Loading the required functions
import pandas as pd
import os
import numpy as np

# I. Extract & Transform

In [2]:
# Define the path using a raw string and expand the ~ to the full path
file_path = os.path.expanduser(r'~\Documents\GitHub\E-Commerce_SupplyChain_DataAnalysis\data\raw\DataCoSupplyChainDataset.csv')

# Load the dataset with the specified encoding
df = pd.read_csv(file_path, encoding='Windows-1252')
df.head()


,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [3]:
print(df.nunique())

Type                                  4
Days for shipping (real)              7
Days for shipment (scheduled)         4
Benefit per order                 21998
Sales per customer                 2927
Delivery Status                       4
Late_delivery_risk                    2
Category Id                          51
Category Name                        50
Customer City                       563
Customer Country                      2
Customer Email                        1
Customer Fname                      782
Customer Id                       20652
Customer Lname                     1109
Customer Password                     1
Customer Segment                      3
Customer State                       46
Customer Street                    7458
Customer Zipcode                    995
Department Id                        11
Department Name                      11
Latitude                          11250
Longitude                          4487
Market                                5


## Initial cleaning: drop column, trim whitespace, correct column type, standardize column name

In [4]:
newdf = df.drop(columns = ["Product Status", 'Product Description','Customer Country'])
newdf.nunique()

Type                                  4
Days for shipping (real)              7
Days for shipment (scheduled)         4
Benefit per order                 21998
Sales per customer                 2927
Delivery Status                       4
Late_delivery_risk                    2
Category Id                          51
Category Name                        50
Customer City                       563
Customer Email                        1
Customer Fname                      782
Customer Id                       20652
Customer Lname                     1109
Customer Password                     1
Customer Segment                      3
Customer State                       46
Customer Street                    7458
Customer Zipcode                    995
Department Id                        11
Department Name                      11
Latitude                          11250
Longitude                          4487
Market                                5
Order City                         3597


In [5]:
# Trim white spaces from all string columns
newdf = newdf.map(lambda x: x.strip() if isinstance(x, str) else x)

# Standardize column names: make all lowercase and replace spaces with underscores
newdf.columns = newdf.columns.str.lower().str.replace(' ', '_')


In [6]:
# checking null values
null_columns = newdf.isna().sum()
null_columns = null_columns[null_columns > 0]
print(null_columns)

customer_lname           8
customer_zipcode         3
order_zipcode       155679
dtype: int64


In [7]:
# Fill missing values with a placeholder
newdf[['customer_zipcode', 'order_zipcode']] = newdf[['customer_zipcode', 'order_zipcode']].fillna(0)

# Remove the decimal part and convert to string
newdf[['customer_zipcode', 'order_zipcode']] = newdf[['customer_zipcode', 'order_zipcode']].astype(float).astype(int).astype(str)

In [8]:
# Replace common placeholders with NaN (pandas treats NaN as NULL)
newdf.replace(['', ' ', 'NaN', 'NULL', 'N/A', 'None', '?', '-', '0', 'Unknown'], pd.NA, inplace=True)

In [9]:
# Check for common placeholders null
print(newdf.isin(['', ' ', 'NaN', 'NULL', 'N/A', 'None', '?', '-', '0', 'Unknown']).sum())

type                             0
days_for_shipping_(real)         0
days_for_shipment_(scheduled)    0
benefit_per_order                0
sales_per_customer               0
delivery_status                  0
late_delivery_risk               0
category_id                      0
category_name                    0
customer_city                    0
customer_email                   0
customer_fname                   0
customer_id                      0
customer_lname                   0
customer_password                0
customer_segment                 0
customer_state                   0
customer_street                  0
customer_zipcode                 0
department_id                    0
department_name                  0
latitude                         0
longitude                        0
market                           0
order_city                       0
order_country                    0
order_customer_id                0
order_date_(dateorders)          0
order_id            

In [10]:
# Identify duplicate rows based on 'order_id'
duplicates = newdf[newdf.duplicated(subset=['order_id','order_item_id'], keep=False)]

In [11]:
# Convert the columns to datetime
newdf['order_date_(dateorders)'] = pd.to_datetime(newdf['order_date_(dateorders)'], format='%m/%d/%Y %H:%M')
newdf['shipping_date_(dateorders)'] = pd.to_datetime(newdf['shipping_date_(dateorders)'], format='%m/%d/%Y %H:%M')

# Convert the datetime columns to ISO 8601 format
newdf['order_date_(dateorders)'] = newdf['order_date_(dateorders)'].dt.strftime('%Y-%m-%d %H:%M:%S')
newdf['shipping_date_(dateorders)'] = newdf['shipping_date_(dateorders)'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [12]:
# Count the number of float columns
num_float_columns = len(newdf.select_dtypes(include=['float']).columns)

# Count the number of integer columns
num_int_columns = len(newdf.select_dtypes(include=['int']).columns)

print(f"Number of float columns: {num_float_columns}")
print(f"Number of integer columns: {num_int_columns}")

Number of float columns: 12
Number of integer columns: 13


In [13]:
# Update 'customer_state' column where the value is '91732' or '95758' to 'CA'
newdf['customer_state'] = newdf['customer_state'].replace(['91732', '95758'], 'CA')

# Update the 'market' column, replacing 'LATAM' with 'Latin America' and 'USCA' with 'North America'
newdf['market'] = newdf['market'].replace({
    'LATAM': 'Latin America',
    'USCA': 'North America'
})

# Update the 'order_city' column with correct names
newdf['order_city'] = newdf['order_city'].replace({
    'Aew?l-li': 'Aewŏl-li',
    'Cox’s B?z?r': 'Cox’s Bāzār',
    'Klaip?da': 'Klaipėda',
    'Bra?ov': 'Brașov',
    'Gy?r': 'Győr',
    'Kahramanmara?': 'Kahramanmaraş'
})

In [14]:
# Mapping of Spanish country names to English country names
country_mapping = {
    'Afganistán': 'Afghanistan', 'Alemania': 'Germany', 'Arabia Saudí': 'Saudi Arabia', 'Argelia': 'Algeria',
    'Azerbaiyán': 'Azerbaijan', 'Bangladés': 'Bangladesh', 'Baréin': 'Bahrain', 'Bélgica': 'Belgium',
    'Belice': 'Belize', 'Benín': 'Benin', 'Bielorrusia': 'Belarus', 'Bolivia': 'Bolivia',
    'Bosnia y Herzegovina': 'Bosnia and Herzegovina', 'Botsuana': 'Botswana', 'Brasil': 'Brazil',
    'Bulgaria': 'Bulgaria', 'Burkina Faso': 'Burkina Faso', 'Burundi': 'Burundi', 'Bután': 'Bhutan',
    'Camboya': 'Cambodia', 'Camerún': 'Cameroon', 'Canada': 'Canada', 'Chad': 'Chad', 'Chile': 'Chile',
    'China': 'China', 'Chipre': 'Cyprus', 'Colombia': 'Colombia', 'Corea del Sur': 'South Korea',
    'Costa de Marfil': 'Ivory Coast', 'Costa Rica': 'Costa Rica', 'Croacia': 'Croatia', 'Cuba': 'Cuba',
    'Dinamarca': 'Denmark', 'Ecuador': 'Ecuador', 'Egipto': 'Egypt', 'El Salvador': 'El Salvador',
    'Emiratos Árabes Unidos': 'United Arab Emirates', 'Eritrea': 'Eritrea', 'Eslovaquia': 'Slovakia',
    'Eslovenia': 'Slovenia', 'España': 'Spain', 'Estados Unidos': 'United States', 'Estonia': 'Estonia',
    'Etiopía': 'Ethiopia', 'Filipinas': 'Philippines', 'Finlandia': 'Finland', 'Francia': 'France',
    'Gabón': 'Gabon', 'Georgia': 'Georgia', 'Ghana': 'Ghana', 'Grecia': 'Greece', 'Guadalupe': 'Guadeloupe',
    'Guatemala': 'Guatemala', 'Guayana Francesa': 'French Guiana', 'Guinea': 'Guinea',
    'Guinea Ecuatorial': 'Equatorial Guinea', 'Guinea-Bissau': 'Guinea-Bissau', 'Guyana': 'Guyana',
    'Haití': 'Haiti', 'Honduras': 'Honduras', 'Hong Kong': 'Hong Kong', 'Hungría': 'Hungary', 'India': 'India',
    'Indonesia': 'Indonesia', 'Irak': 'Iraq', 'Irán': 'Iran', 'Irlanda': 'Ireland', 'Israel': 'Israel',
    'Italia': 'Italy', 'Japón': 'Japan', 'Jordania': 'Jordan', 'Kazajistán': 'Kazakhstan', 'Kenia': 'Kenya',
    'Kirguistán': 'Kyrgyzstan', 'Kuwait': 'Kuwait', 'Laos': 'Laos', 'Lesoto': 'Lesotho', 'Líbano': 'Lebanon',
    'Liberia': 'Liberia', 'Libia': 'Libya', 'Lituania': 'Lithuania', 'Luxemburgo': 'Luxembourg',
    'Macedonia': 'North Macedonia', 'Madagascar': 'Madagascar', 'Malasia': 'Malaysia', 'Mali': 'Mali',
    'Marruecos': 'Morocco', 'Martinica': 'Martinique', 'Mauritania': 'Mauritania', 'México': 'Mexico',
    'Moldavia': 'Moldova', 'Mongolia': 'Mongolia', 'Montenegro': 'Montenegro', 'Mozambique': 'Mozambique',
    'Myanmar (Birmania)': 'Myanmar', 'Namibia': 'Namibia', 'Nepal': 'Nepal', 'Nicaragua': 'Nicaragua',
    'Níger': 'Niger', 'Nigeria': 'Nigeria', 'Noruega': 'Norway', 'Nueva Zelanda': 'New Zealand', 'Omán': 'Oman',
    'Países Bajos': 'Netherlands', 'Pakistán': 'Pakistan', 'Panamá': 'Panama', 'Papúa Nueva Guinea': 'Papua New Guinea',
    'Paraguay': 'Paraguay', 'Perú': 'Peru', 'Polonia': 'Poland', 'Portugal': 'Portugal', 'Qatar': 'Qatar',
    'Reino Unido': 'United Kingdom', 'República Centroafricana': 'Central African Republic', 'República Checa': 'Czech Republic',
    'República de Gambia': 'Gambia', 'República del Congo': 'Republic of the Congo', 'República Democrática del Congo': 'Democratic Republic of the Congo',
    'República Dominicana': 'Dominican Republic', 'Ruanda': 'Rwanda', 'Rumania': 'Romania', 'Rusia': 'Russia',
    'Sáhara Occidental': 'Western Sahara', 'Senegal': 'Senegal', 'Serbia': 'Serbia', 'Sierra Leona': 'Sierra Leone',
    'Singapur': 'Singapore', 'Siria': 'Syria', 'Somalia': 'Somalia', 'Sri Lanka': 'Sri Lanka', 'Suazilandia': 'Eswatini',
    'SudAfrica': 'South Africa', 'Sudán': 'Sudan', 'Sudán del Sur': 'South Sudan', 'Suecia': 'Sweden', 'Suiza': 'Switzerland',
    'Surinam': 'Suriname', 'Tailandia': 'Thailand', 'Taiwán': 'Taiwan', 'Tanzania': 'Tanzania', 'Tayikistán': 'Tajikistan',
    'Togo': 'Togo', 'Trinidad y Tobago': 'Trinidad and Tobago', 'Túnez': 'Tunisia', 'Turkmenistán': 'Turkmenistan',
    'Turquía': 'Turkey', 'Ucrania': 'Ukraine', 'Uganda': 'Uganda', 'Uruguay': 'Uruguay', 'Uzbekistán': 'Uzbekistan',
    'Venezuela': 'Venezuela', 'Vietnam': 'Vietnam', 'Yemen': 'Yemen', 'Yibuti': 'Djibouti', 'Zambia': 'Zambia',
    'Zimbabue': 'Zimbabwe'
}

# Update the 'order_country' column with the correct English names
newdf['order_country'] = newdf['order_country'].replace(country_mapping)


In [18]:
output_dir = r"C:\Users\PC\Documents\GitHub\E-Commerce_SupplyChain_DataAnalysis\data\interim"
output_file = os.path.join(output_dir, "cleaned_data.csv")

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save the cleaned data
newdf.to_csv(output_file, index=False)
print(f"File saved at: {output_file}")


File saved at: C:\Users\PC\Documents\GitHub\E-Commerce_SupplyChain_DataAnalysis\data\interim\cleaned_data.csv


In [20]:
stats = newdf.describe()
stats

,days_for_shipping_(real),days_for_shipment_(scheduled),benefit_per_order,sales_per_customer,late_delivery_risk,category_id,customer_id,customer_zipcode,department_id,latitude,...,order_item_product_price,order_item_profit_ratio,order_item_quantity,sales,order_item_total,order_profit_per_order,order_zipcode,product_card_id,product_category_id,product_price
count,180519.000000,180519.000000,180519.000000,180519.000000,180519.000000,180519.000000,180519.000000,180516.000000,180519.000000,180519.000000,...,180519.000000,180519.000000,180519.000000,180519.000000,180519.000000,180519.000000,24840.000000,180519.000000,180519.000000,180519.000000
mean,3.497654,2.931847,21.974989,183.107609,0.548291,31.851451,6691.379495,35921.126914,5.443460,29.719955,...,141.232550,0.120647,2.127638,203.772096,183.107609,21.974989,55426.132327,692.509764,31.851451,141.232550
std,1.623722,1.374449,104.433526,120.043670,0.497664,15.640064,4162.918106,37542.461122,1.629246,9.813646,...,139.732492,0.466796,1.453451,132.273077,120.043670,104.433526,31919.279101,336.446807,15.640064,139.732492
min,0.000000,0.000000,-4274.979980,7.490000,0.000000,2.000000,1.000000,603.000000,2.000000,-33.937553,...,9.990000,-2.750000,1.000000,9.990000,7.490000,-4274.979980,1040.000000,19.000000,2.000000,9.990000
25%,2.000000,2.000000,7.000000,104.379997,0.000000,18.000000,3258.500000,725.000000,4.000000,18.265432,...,50.000000,0.080000,1.000000,119.980003,104.379997,7.000000,23464.000000,403.000000,18.000000,50.000000
50%,3.000000,4.000000,31.520000,163.990005,1.000000,29.000000,6457.000000,19380.000000,5.000000,33.144863,...,59.990002,0.270000,1.000000,199.919998,163.990005,31.520000,59405.000000,627.000000,29.000000,59.990002
75%,5.000000,4.000000,64.800003,247.399994,1.000000,45.000000,9779.000000,78207.000000,7.000000,39.279617,...,199.990005,0.360000,3.000000,299.950012,247.399994,64.800003,90008.000000,1004.000000,45.000000,199.990005
max,6.000000,4.000000,911.799988,1939.989990,1.000000,76.000000,20757.000000,99205.000000,12.000000,48.781933,...,1999.989990,0.500000,5.000000,1999.989990,1939.989990,911.799988,99301.000000,1363.000000,76.000000,1999.989990
